# 20.03 The ID3 Algorithm

So far decision tree growing with an entropy criterion has been covered.  In doing so how that would actually work got skipped.  This notebook will give more detail into how an algorithm to do that will practically function.  This notebook will cover the ID3 algorithm, which is a popular algorithm for building decision trees.

### The ID3 Algorithm 

ID3 stands for _Iterative Dichotomizer 3_, and is one of the simplest ways to create a decision tree.  It can be generalized into more robust scenarios, but the implementation is based on the framework covered here.  Essentially ID3 goes through every feature to find the most valuable attribute then then splits base on it.  It moves further and further down the tree until it either has a pure class or has met a terminating condition.  The details will be covered below.  

Before you can start working with ID3 there are some requirements for the data in its most basic form.  First, outcomes must be binary.  The simplest version of ID3 is a binary classifier.  The attributes you'll use to build the tree also have to be categorical.  Attributes can have many categories, but they must be known and countable.  If those two criteria are met, then you can build a basic ID3 classifying algorithm.

The other thing you'll need for this is the definition of entropy from 20.02.  Shannon Entropy $H$, is defined as: 

$$H = -\sum_{i=1}^n P(x_i)log_2 P(x_i)$$

For simplicity of calculation, you can make the following transformation to the definition: 

$$H = \sum_{i=1}^n P(x_i)log_2 \frac{1}{P(x_i)}$$

This removes the negative sign up front and will make it easier to implement the formula.

### Calculating Entropy 

Since this algorithm is based on entropy, a quick example will be given on how to calculate it.  Say you have 20 students, and you're trying to classify them as male and female.  The only attribute you have is whether their height is tall, medium, or short.  Of the 20 students, 12 are boys and 8 are girls.  Of the 12 boys, 4 are tall, 6 are medium, and 2 are short.  For the 8 girls, 1 is tall, 2 are medium, and 5 are short.

You are trying to find the entropy, both before and after any rule is applied for being tall.  The initial entropy is just the formula plugged in over both the possible classes of interest:

$$ H = P(male)*log_2 \frac{1}{P(male)} + P(female)*log_2 \frac{1}{P(female)}  $$


$$ = \frac{12}{20}*log_2 \frac{20}{12} + \frac{8}{20}*log_2 \frac{20}{8} = .971 $$

Now apply the rule _height = short_.  You need to calculate the weighted average of the two entropies, one for the short students and one for the non-short students.

$$ H(short) = P(male)*log_2 \frac{1}{P(male)} + P(female)*log_2 \frac{1}{P(female)}  $$

$$ = \frac{2}{7}*log_2 \frac{7}{2} + \frac{5}{7}*log_2 \frac{7}{5} = .863 $$

$$ H(not\_short) = P(male)*log_2 \frac{1}{P(male)} + P(female)*log_2 \frac{1}{P(female)}  $$

$$ = \frac{10}{13}*log_2 \frac{13}{10} + \frac{3}{13}*log_2 \frac{13}{3} = .779 $$

Note that all the probabilities are conditional on the criteria you’re assuming (short or not short).  Now the weighted average of the two is just:

$$ P(short) * H(short) + P(not\_short) * H(not\_short) = \frac{7}{20} * .863 + \frac{13}{20} * .779 = .809 $$

The entropy from this question goes from 0.971 to 0.809.  That is an improvement.  Use the space below to calculated the entropy of the other criteria, calculate for students tall or medium:  


In [21]:
import numpy as np 

from math import log2

def find_H(inputs):
    pM = inputs[1]/inputs[0]
    pF = inputs[2]/inputs[0]
    X1 = pM*log2(1/pM)
    X2 = pF*log2(1/pF)

    H = sum([X1,X2])

    return [H,pM,pF,X1,X2]

# [Students, Male, Female]
inputs_all = [20,12,8]
# [Short,Male_short, Female_short]
inputs_short = [7,2,5]
inputs_short_complement = list(np.array(inputs_all) - np.array(inputs_short))
# [Medium,Male_medium,Female_medium]
inputs_medium = [8,6,2]
inputs_medium_complement = list(np.array(inputs_all) - np.array(inputs_medium))
# [Tall,Male_tall,Female_tall]
inputs_tall = [5,4,1]
inputs_tall_complement = list(np.array(inputs_all) - np.array(inputs_tall))

# Calculate Initial H
H_initial = find_H(inputs_all)

# Find H for short and its complement
H_short = find_H(inputs_short)
H_not_short = find_H(inputs_short_complement)

# Find H for medium and its complement
H_medium = find_H(inputs_medium)
H_not_medium = find_H(inputs_medium_complement)

# Find H for tall and its complement
H_tall = find_H(inputs_tall)
H_not_tall = find_H(inputs_tall_complement)

# Calcuate the entropies for short, medium and tall
entropy_short = ((inputs_short[0]/inputs_all[0]) * H_short[0]) + ((inputs_short_complement[0]/inputs_all[0]) * H_not_short[0])
entropy_medium = ((inputs_medium[0]/inputs_all[0]) * H_medium[0]) + ((inputs_medium_complement[0]/inputs_all[0]) * H_not_medium[0])
entropy_tall = ((inputs_tall[0]/inputs_all[0]) * H_tall[0]) + ((inputs_tall_complement[0]/inputs_all[0]) * H_not_tall[0])

print(f"Entropy Short: {entropy_short} | Entropy Medium: {entropy_medium} | Entropy Tall: {entropy_tall}")

Entropy Short: 0.808669593238176 | Entropy Medium: 0.9245112497836531 | Entropy Tall: 0.9280757477080679


You should have found entropies of .925 for medium and .928 for tall. Both of those entropies are higher. Now, it has been said previously said that you want to prioritize questions with the most information gain. Which one of these would that be?

It would be asking if an individual is short. You should also note that for all possible questions, we're still comparing with the same initial entropy value. So one way of seeing which question has the most information gain is to find the one with the lowest entropy. This should make sense when we think of entropy as uncertainty. The less uncertainty after a question, the more information we gained.

### Pseudocoding the Algorithm

**Pseudocode** is the process of writing the steps and logic you implement in code, but in normal language rather than commands a programming language could execute.  It can be a useful way to chart out how you want to build an algorithim and is a common topic for technical interviews.  Here, pseudocode will be used to expain the ID3 Algorithm.

Here is reasonable pseudocode for ID3, which will then be followed up with an explanation of the steps.  The outcome for this variable will be A or B.  An attribute is denoted a<sub>i</sub>.  A value of that attribute is v<sub>i</sub>

<pre>
Algorithm(Observations, Outcome, Attributes)
    Create a root node.
    If all observations are 'A', label root node 'A' and return.
    If all observations are 'B', label root node 'B' and return.
    If no attributes return the root note labeled with the most common Outcome.
    Otherwise, start:
        For each value v<sub>i</sub> of each attribute a<sub>i</sub>, calculate the entropy.
        The attribute a<sub>i</sub> and value v<sub>i</sub> with the lowest entropy is the best rule.
        The attribute for this node is then a<sub>i</sub>
            Split the tree to below based on the rule a<sub>i</sub> = v<sub>i</sub>
            Observations<sub>v<sub>i</sub></sub> is the subset of observations with value v<sub>i</sub>
            If Observations<sub>v<sub>i</sub></sub> is empty cap with node labeled with most common Outcome
            Else at the new node start a subtree (Observations<sub>v<sub>i</sub></sub>, Target Outcome, Attributes - {a<sub>i</sub>}) and repeat the algorithm
</pre>

Now let's walk through this pseudocode algorithm in plain English piece by piece.

First you create a root node. Simple enough, you have to start with something.

The next two lines say that if you're already exclusively one class, just label with that class and you're done. Similarly the following line says that if you don't have any attributes left you're also done, labeling with the most common outcome.

Then we get into the _real_ algorithm. First you have to find the best attribute by calculating entropies for all possible values. The attribute value pair with the lowest entropy is then the best attribute and the attribute you give to the node.

You use that rule to split the node, creating subsets of observations. There are then two new nodes, each with a subset of the observations corresponding to the rule. If obsevations are null then label with the dominant outcome.

Otherwise at each new node start the algorithm again.

This is how a decision tree would actually function. Understanding this should give you some insight into how this algorithm works and reveals several attributes of the algorithm. Firstly, the solution is not necessarily optimal. The tree can get stuck in local optima, just like with the gradient descent algorithm. It also has no way to work backwards if it finds itself in an informationless space. You can add criteria to make it stop before the tree has grown to run out of attributes or all leaf nodes are single classes.

### DRILL:

Look over the code for [this real](https://github.com/NinjaSteph/DecisionTree/blob/master/sna2111_DecisionTree/DecisionTree.py) ID3 Algorithm in python. Note how well the author breaks up functionality into individual, reusable, well-named helper functions. See if you can match our pseudocode steps to the code in this example.